# Cookbook de EDA

## 0. Setup básico

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("data.csv")   # o el dataset que uses
```

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data
df = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/data-preprocessing-project-tutorial/main/AB_NYC_2019.csv")   # o el dataset que uses

## Checklist general de EDA (para pegar en la pared)

1. Entender el problema y cada fila.
2. Mirada rápida a los datos.
3. Revisar estructura y tipos.
4. Buscar valores faltantes.
5. Buscar duplicados.
6. Analizar variables numéricas.
7. Analizar variables categóricas.
8. Analizar fechas/tiempo (si hay).
9. Analizar texto libre (si hay).
10. Ver relación con la variable objetivo.
11. Buscar correlaciones y multicolinealidad.
12. Buscar outliers y valores raros.
13. Pensar en calidad del dato y “sentido de negocio”.
14. Anotar conclusiones y decisiones (drop, imputar, transformar, etc.).





## Paso 1: Entender el problema y la fila

**Objetivo:** saber qué significa cada fila y qué queremos predecir.

**Pasos (sin código):**

* Preguntar:

  * ¿Qué representa una fila? ¿Un cliente, una compra, una medición de sensor…?
  * ¿Qué queremos predecir o explicar? → **target** (columna objetivo).
  * ¿Qué columnas solo se conocen después del resultado? (posible leakage)

**Nota:**
Si no saben qué significa una columna → *anotar la duda* y no usarla para modelar hasta entenderla.




## Paso 2: Mirada rápida al dataset

**Objetivo:** tener una idea general del tamaño y forma.

```python
df.shape          # (filas, columnas)
df.head()         # primeras filas
df.tail()         # últimas filas
df.sample(5, random_state=0)  # filas aleatorias
```

**Qué mirar:**

* ¿Hay columnas raras (todo 0, todo igual, códigos raros)?
* ¿El tamaño tiene sentido (no son 10 filas cuando esperabas miles)?




## Paso 3: Estructura y tipos de datos

**Objetivo:** saber si las columnas están con el tipo correcto.

```python
df.info()
df.dtypes
```

**Trucos:**

* IDs → normalmente `object` o `int` pero **no se usan para modelos** (solo como identificadores).
* Fechas que aparecen como texto → convertir a datetime:

```python
df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")
```

* Variables numéricas que en realidad son categorías (ej. “1=Bajo, 2=Medio, 3=Alto”) → tratarlas como categóricas.




## Paso 4: Valores faltantes (NaN)

**Objetivo:** ver dónde falta información.

```python
df.isna().sum()            # por columna
(df.isna().mean()*100)     # % de NaN por columna
```

**Qué mirar:**

* Columnas con **muchos NaN** (ej. > 40–50%).
* Columnas con **0 NaN** (quizás ya fueron imputadas o son obligatorias).

**Decisiones típicas (para explicar simple):**

* **Pocos NaN** → rellenar con:

  * numéricos: media/mediana o un valor especial (ej. -1).
  * categóricos: valor “Desconocido”.
* **Muchísimos NaN** → considerar eliminar la columna.
* NaN que tienen significado (ej. “no tiene segunda dirección”) → inventar categoría “No aplica”.

Ejemplo:

```python
# numérica
df["edad"] = df["edad"].fillna(df["edad"].median())

# categórica
df["ciudad"] = df["ciudad"].fillna("Desconocido")
```



## Paso 5: Duplicados

**Objetivo:** ver si hay filas repetidas.

```python
df.duplicated().sum()
df[df.duplicated()].head()
```

Para eliminarlos:

```python
df = df.drop_duplicates()
```

**Nota:**
A veces no es “toda la fila” duplicada, sino un ID repetido → hay que decidir si es un error o son eventos diferentes.




## Paso 6.1: Variables numéricas (visión general)

**Objetivo:** ver rangos, medias y valores raros.

```python
num_cols = df.select_dtypes(include=["int64", "float64"]).columns
df[num_cols].describe()
```

**Trucos:**

* Mirar si hay valores imposibles (edad negativa, salario 1e9, etc.).
* Ver si hay columnas casi constantes (máx ≈ mín).




## Paso 6.2: Variables numéricas (distribuciones)

**Objetivo:** ver la forma de cada variable (sesgo, cola larga, etc.).

Ejemplo con una columna:

```python
col = "edad"
df[col].hist(bins=30)
plt.xlabel(col)
plt.ylabel("Frecuencia")
plt.show()
```

**Qué ver:**

* **Simétrico**: parecido a una campana.
* **Sesgado a la derecha**: muchos valores pequeños y pocos muy grandes (típico en ingresos, tiempos, montos).
* **Sesgado a la izquierda**: al revés.

**Ideas de acciones simples:**

* Distribución muy sesgada → considerar logaritmo:
  `df["ingreso_log"] = np.log1p(df["ingreso"])`

Transformamos para que los modelos puedan entender mejor. No queremos colas extremas dominantes o escalas desproporcionadas. Aunque estas transformaciones no son necesarias para todos los modelos.




## Paso 7.1: Variables categóricas (conteos)

**Objetivo:** ver cuántas categorías hay y cuáles son las más frecuentes.

```python
cat_cols = df.select_dtypes(include=["object", "category"]).columns

for col in cat_cols:
    print(f"\n--- {col} ---")
    print(df[col].value_counts().head(10))
    print("N categorías:", df[col].nunique())
```

**Trucos:**

* Columnas con **demasiadas categorías** (ej. texto libre, IDs) → cuidado para modelos.
* Categorías muy raras → se pueden agrupar en “Otros”.




## Paso 7.2: Variables categóricas (gráficos simples)

Ejemplo: distribución de una categoría.

```python
col = "ciudad"
df[col].value_counts().head(10).plot(kind="bar")
plt.ylabel("Cuenta")
plt.show()
```

**Qué mirar:**

* Categorías dominantes (ej. una ciudad es 90% de los casos).
* Categorías casi sin datos (quizás no valga la pena usarlas).




## Paso 8: Fechas y tiempo

**Objetivo:** aprovechar columnas de fecha.

Asumimos que ya convertiste la columna:

```python
df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")
```

**Crear columnas derivadas fáciles:**

```python
df["anio"] = df["fecha"].dt.year
df["mes"] = df["fecha"].dt.month
df["dia_semana"] = df["fecha"].dt.day_name()
```

**Analizar evolución en el tiempo:**

```python
df_por_mes = df.groupby(["anio", "mes"])["target"].mean()  # o count
df_por_mes.plot()
plt.ylabel("Target medio")
plt.show()
```

**Nota:** ver si hay **tendencias** o **estacionalidad** (por meses, días de semana).




## Paso 9: Texto libre (muy básico)

**Objetivo:** tener idea de qué trae una columna de texto.

```python
df["comentario"].str.len().describe()  # longitud de texto
df["comentario"].head()
```

Trucos muy simples:

* Crear una columna con la **longitud del texto**:

  ```python
  df["len_comentario"] = df["comentario"].str.len()
  ```
* Contar cuántos están vacíos o muy cortos:

  ```python
  (df["comentario"].str.len() == 0).mean()
  ```




## Paso 10: Relación con la variable objetivo (clasificación)

### 10.1. Numérica vs target binario (0/1)

**Objetivo:** ver si la variable ayuda a separar clases.

```python
col = "edad"
df.groupby("target")[col].describe()
```

**Explicación simple:**
Comparar la media de `edad` para target=0 y target=1.
Si son muy diferentes, esa variable probablemente sea útil.



### 10.2. Categoría vs target binario

```python
col = "ciudad"
tabla = (
    df.groupby(col)["target"]
      .mean()
      .sort_values(ascending=False)
      .head(10)
)
print(tabla)
```

**Qué mirar:**
Ciudades con tasa de target mucho más alta o más baja → categorías importantes.





## Paso 11: Correlaciones (numéricas)

**Objetivo:** ver qué variables numéricas se parecen (se mueven juntas).

```python
corr = df[num_cols].corr()
corr["target"].sort_values(ascending=False)
```

**Que significa:**

* Correlación cerca de **1**: suben y bajan juntas.
* Cerca de **-1**: una sube cuando la otra baja.
* Cerca de **0**: no hay relación lineal clara.

También ver **correlaciones entre variables predictoras**:

* Si dos columnas están casi perfectamente correlacionadas → quizás una sobra.




## Paso 12: Outliers (valores extremos)

### 12.1. Por regla simple

```python
col = "ingreso"
df[col].describe()
```

Luego:

```python
# Filas con valores muy grandes
df[df[col] > df[col].quantile(0.99)].head()
```

### 12.2. Boxplot simple

```python
df[[col]].boxplot()
plt.show()
```

**Nota:**

* Un outlier no siempre es un error.
* Preguntar: ¿es un error de carga o un cliente realmente enorme?
* Opciones simples: recortar (cap), transformar (log), o dejarlos pero avisar al modelo.




## Paso 13.1: Columnas inútiles o sospechosas

**Checklist simple:**

* Columnas **idénticas** o muy similares entre sí.
* Columnas con **un solo valor** (sin variabilidad).
* IDs puros (no explican nada, solo identifican).
* Columnas que se conocen **después** del resultado (leakage).

Código rápido:

```python
# columnas casi constantes
n = len(df)
for col in df.columns:
    freq = df[col].value_counts(normalize=True, dropna=False).iloc[0]
    if freq > 0.99:
        print("Casi constante:", col, f"({freq:.2%} mismo valor)")
```




## Paso 13.2: Data leakage (explicación simple)

**Objetivo:** evitar “hacer trampa”.

**Ejemplos:**

* Predecir si un cliente dejará la empresa usando una columna “fecha_de_baja”.
* Predecir ventas del mes usando la columna “ventas_del_mes” 🤦.

**Regla de oro:**

> Si la columna no estaría disponible en el momento real de hacer la predicción, no puede usarse.




## Paso 14: Crear reporte y/o pipeline

Anotar conclusiones y decisiones (drop, imputar, transformar, etc.) para poder replicar en otros datasets (train, test o actualizaciones) o experimentos.

## Paso 15 (bonus): Reporte rápido automático

- [sweetviz](https://colab.research.google.com/github/amiune/freecodingtour/blob/main/cursos/espanol/datascience/analysis/A1.2_eda_sweetviz.ipynb)
- [pandas_dq](https://colab.research.google.com/github/amiune/freecodingtour/blob/main/cursos/espanol/datascience/analysis/A3.3_pandas_dq.ipynb)
-[mas...](https://colab.research.google.com/github/amiune/freecodingtour/blob/main/cursos/espanol/datascience/analysis/A0_intro.ipynb#scrollTo=48149491-163e-4f79-a52a-91e027f85bf5)

# Paso 16 (extra bonus): Ver distintos analisis en kaggle

- https://www.kaggle.com/datasets/saritas95/nyc-airbnb-insights-a-data-analysis
- https://www.kaggle.com/code/phongphmlm/airbnb-new-york-city-2019
- https://www.kaggle.com/code/dgomonov/data-exploration-on-nyc-airbnb
- https://www.kaggle.com/code/chbenitez/airbnb-nyc-an-lisis-y-preprocesamiento-de-datos/notebook
- https://www.kaggle.com/code/duygut/airbnb-nyc-price-prediction